In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import preprocess_input
from tensorflow.keras.applications import ResNet50, ResNet50V2, Xception
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, AveragePooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, Adadelta, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/MyDrive/PlateDataset/')
!unzip Dataset.zip

In [ ]:
IMG_WIDTH, IMG_HEIGHT = 224, 224
EPOCHS = 60
BATCH_SIZE= 60
n_classes = 3

In [ ]:
DATA_PATH = '/content/drive/MyDrive/PlateDataset/Dataset/'

In [ ]:
def recall_m(y_true, y_pred):
    K = tf.keras.backend
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    K = tf.keras.backend
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    K = tf.keras.backend
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
train_datagen_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                      shear_range=0.25,
                                      width_shift_range=0.25,
                                      height_shift_range=0.25,
                                      zoom_range=0.2,
                                      rotation_range=40,
                                      validation_split=0.2)

train_generator_aug  = train_datagen_aug.flow_from_directory(DATA_PATH,
                                                              target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                              batch_size=BATCH_SIZE,
                                                              class_mode='categorical',
                                                              subset='training')

validation_generator  = validation_datagen.flow_from_directory(DATA_PATH,
                                                              target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                              batch_size=BATCH_SIZE,
                                                              class_mode='categorical',
                                                              subset='validation')

Found 2183 images belonging to 3 classes.
Found 544 images belonging to 3 classes.


In [ ]:
def build_model():
  baseModel = Xception(
        include_top=False,
        weights="imagenet",
        input_tensor=Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        pooling='avg'
    )
    
  # for layer in baseModel.layers:
  #     layer.trainable = False
  # for layer in baseModel.layers[-40:]:
  #     layer.trainable = True

  headModel = baseModel.output
  headModel = Dropout(0.2)(headModel)
  headModel = Dense(1024, activation="relu")(headModel)
  headModel = Dropout(0.2)(headModel)
  headModel = Dense(1024, activation="relu")(headModel)
  headModel = Dropout(0.2)(headModel)
  headModel = Dense(512, activation="relu")(headModel)
  headModel = Dropout(0.2)(headModel)
  headModel = Dense(512, activation="relu")(headModel)
  headModel = Dropout(0.2)(headModel)
  headModel = Dense(n_classes, activation="softmax")(headModel)
  
  model = Model(inputs=baseModel.input, outputs=headModel)

  return model

In [ ]:
# learning_rate = 0.0001
weights = '/content/drive/MyDrive/PlateDatasetModels/Xception_weights.h5'
callbacks = [ 
            #  EarlyStopping(monitor='val_f1_m', patience=20, verbose=0), 
              ModelCheckpoint(weights, monitor='val_f1_m', save_best_only=True, verbose=0, mode='max')
              # ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)]
            ]
loss = CategoricalCrossentropy()
# optimizer = SGD(lr=1e-2, momentum=0.9, decay=1e-2/40)
optimizer = Adam()
model = build_model()
model.compile(
    loss=loss, 
    optimizer=optimizer, 
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

history = model.fit(
    train_generator_aug, 
    validation_data=validation_generator, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    callbacks=callbacks
)

Epoch 1/60
35/35 [==============================] - 54s 1s/step - loss: 0.6796 - accuracy: 0.7282 - f1_m: 0.6457 - precision_m: 0.7864 - recall_m: 0.5554 - val_loss: 27.8160 - val_accuracy: 0.6893 - val_f1_m: 0.6892 - val_precision_m: 0.6892 - val_recall_m: 0.6892
Epoch 2/60
35/35 [==============================] - 51s 1s/step - loss: 0.3076 - accuracy: 0.8801 - f1_m: 0.8786 - precision_m: 0.9033 - recall_m: 0.8561 - val_loss: 1.2178 - val_accuracy: 0.8658 - val_f1_m: 0.8661 - val_precision_m: 0.8731 - val_recall_m: 0.8594
Epoch 3/60
35/35 [==============================] - 51s 1s/step - loss: 0.2688 - accuracy: 0.8930 - f1_m: 0.8870 - precision_m: 0.9058 - recall_m: 0.8703 - val_loss: 0.5956 - val_accuracy: 0.8382 - val_f1_m: 0.8430 - val_precision_m: 0.8513 - val_recall_m: 0.8351
Epoch 4/60
35/35 [==============================] - 52s 1s/step - loss: 0.2334 - accuracy: 0.9165 - f1_m: 0.9141 - precision_m: 0.9217 - recall_m: 0.9070 - val_loss: 5.7203 - val_accuracy: 0.8640 - val_f1_m:

In [ ]:
best_model = tf.keras.models.load_model(weights, custom_objects={'f1_m':f1_m, 'precision_m':precision_m, 'recall_m':recall_m})
best_model.compile(
    loss=loss, 
    optimizer=optimizer, 
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

In [ ]:
best_model.evaluate(validation_generator, batch_size=batch_size)

9/9 [==============================] - 8s 704ms/step - loss: 0.1535 - accuracy: 0.9770 - f1_m: 0.9749 - precision_m: 0.9772 - recall_m: 0.9727


[0.14328277111053467,
 0.9761029481887817,
 0.9765419363975525,
 0.9774029850959778,
 0.9756944179534912]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

test_generator = train_datagen.flow_from_directory(
    DATA_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
  )

Y_pred = best_model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['0', '1', '2']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Found 544 images belonging to 3 classes.
Confusion Matrix
[[372   2   1]
 [ 10  45   1]
 [  1   0 112]]
Classification Report
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       375
           1       0.96      0.80      0.87        56
           2       0.98      0.99      0.99       113

    accuracy                           0.97       544
   macro avg       0.97      0.93      0.95       544
weighted avg       0.97      0.97      0.97       544

